In [ ]:
# simulator部分-数据要求：
# bayesprism： 无特殊要求
# music（非层次化反解）： 需要添加样本信息，复制所有细胞，原始数据sample标为1，复制的标为2，参考/home/rui/simulator/data1/sc_ct_musictest

# simulation 数据存储
# 每个测试生成的bulk和sc及满足多个方法要求的sc_ct matrix保存在该测试的文件夹中、
# benchmark以文件夹路径为参数依次读入和运行多个deconvolution method

# benchmark code - function注解
# 1. bayesprism：参数（文件夹路径，m），
    #  其中m=0：使用bayesprism找的marker；m=1：使用提供的marker list进行反解
    # 读入的数据:sc_m, sc_ct, bulk，如果使用提供的marker list，则sc_m为marker筛选之后的matrix
#2.

In [1]:
import rpy2.robjects as ro
import numpy as np
import pandas as pd
r=ro.r

### BeysPrism

In [2]:
### function: deconvolution with bayesprism:
###         变量：sc, bulk, method
folder_path = "/home/rui/simulator/data2/"

r('''
    bayesprism <- function(folder_path,m){
        library(BayesPrism)
        
        sc.dat <- read.csv(file.path(folder_path,"sc_m5"))
        cell.type.labels <- read.csv(file.path(folder_path,"sc_ct5"))
        bk.dat <- read.csv("/home/rui/simulator/data2/bulk5")
        sc.dat <- sc.dat[,-1]
        cell.type.labels <- cell.type.labels[,-1]
        #cell.state.labels <- cell.state.labels[,-1]
        bk.dat <- bk.dat[,-1]

        if(m == 0){
            diff.exp.stat <- get.exp.stat(sc.dat=sc.dat[,colSums(sc.dat>0)>3],# filter genes to reduce memory use
                                cell.type.labels=cell.type.labels,
                                cell.state.labels=cell.type.labels,
                                psuedo.count=0.1, #a numeric value used for log2 transformation. =0.1 for 10x data, =10 for smart-seq.
                                cell.count.cutoff=50, # a numeric value to exclude cell state with number of cells fewer than this valu
                                n.cores=1 #number of threads
            )

            sc.dat.filtered.pc.sig <- select.marker (sc.dat=sc.dat,
                                            stat=diff.exp.stat,
                                            pval.max=0.01,
                                            lfc.min=0.1
            )
            myPrism <- new.prism(
            reference=sc.dat.filtered.pc.sig,
            mixture=bk.dat,
            input.type="count.matrix",
            cell.type.labels = cell.type.labels,
            cell.state.labels = cell.type.labels,
            key=NULL,
            outlier.cut=0.01,
            outlier.fraction=0.1
            )

            bp.res <- run.prism(prism = myPrism, n.cores=50)

            theta <- get.fraction (bp=bp.res,
                                which.theta="final",
                                state.or.type="type"
            )
            return(theta)
        }   

  
        if(m == 1){
            sc.dat.selected <- sc.dat

            myPrism <- new.prism(
            reference=sc.dat.selected,
            mixture=bk.dat,
            input.type="count.matrix",
            cell.type.labels = cell.type.labels,
            cell.state.labels = cell.type.labels,
            key=NULL,
            outlier.cut=0.01,
            outlier.fraction=0.1,
            )
            bp.res <- run.prism(prism = myPrism, n.cores=50)
            theta <- get.fraction (bp=bp.res,
                                which.theta="final",
                                state.or.type="type"
            )
            return(theta)
        }
    } 
''')    



### MuSiC

In [115]:
%%capture capture_output
test=r['bayesprism'](folder_path,1)
bp = pd.DataFrame(np.array(test))


In [4]:
# 
folder_path = "/home/rui/simulator/data1/"
r('''
    music <- function(folder_path,m){
        library(MuSiC)
        library(SingleCellExperiment)
        sc.dat <- read.csv(file.path(folder_path,"sc_m_music"))
        sc.dat <- sc.dat[,-1]
        sc.dat <- t(sc.dat)
        sc.dat <- as.data.frame(sc.dat)
        sce <- SingleCellExperiment(assays = list(counts = sc.dat))
        cell.type.labels <- read.csv(file.path(folder_path,"sc_ct_musictest"))
        cell.type.labels <- cell.type.labels[,-1]
        cell.type.labels <- as.data.frame(cell.type.labels)
        rownames(cell.type.labels) <- paste0("cell_", 1:nrow(cell.type.labels))
        colData(sce) <- DataFrame(cell.type.labels)
        counts <- counts(sce)

        bk.dat <- read.csv(file.path(folder_path,"bulk1"))
        bulk.matrix <- t(bk.dat[,-1])
        sample.label <- bk.dat$X
        sample.label<-as.data.frame(sample.label)
        sl<-new("AnnotatedDataFrame", data=sample.label)
        bk.set<-ExpressionSet(bulk.matrix,phenoData = sl)
        bulk.mtx <- exprs(bk.set)

        Est.prop.test = music_prop(bulk.mtx = bulk.mtx, sc.sce  = sce, clusters = 'cell_type',
                                    samples = 'sampleID',select.ct = unique(cell.type.labels[,1]))

        result <- Est.prop.test[["Est.prop.weighted"]]
        return(result)
    } 
''')    

In [5]:
%%capture capture_output
elm= r['music'](folder_path,1)
music = pd.DataFrame(np.array(elm))

### Linseed

In [11]:
folder_path = "/home/rui/simulator/data1/"
r('''
    linseed <- function(folder_path){
    library(linseed)
    bk.dat <- read.csv(file.path(folder_path,"bulk1"))

    bk.dat <- bk.dat[,-1]
    bulk <- t(bk.dat)
    lo <- LinseedObject$new(bulk, samples=0:ncol(bulk))
    lo$calculatePairwiseLinearity()
    lo$calculateSpearmanCorrelation()
    lo$calculateSignificanceLevel(100)
    #lo$significancePlot(0.01)
    lo$filterDatasetByPval(0.01)

    #lo$svdPlot()
    cell.type.labels <- read.csv(file.path(folder_path,"sc_ct_musictest"))
    cell.type.labels <- cell.type.labels[,-1]

    lo$setCellTypeNumber(length(unique(cell.type.labels[,1])))
    lo$project("full") # projecting full dataset
    lo$projectionPlot(color="filtered")

    lo$project("filtered")
    lo$smartSearchCorners(dataset="filtered", error="norm")
    lo$deconvolveByEndpoints()

    return(lo[["proportions"]])
    }
''') 

In [ ]:
linseed = r['linseed'](folder_path)
linseed = pd.DataFrame(np.array(linseed))

### 

In [ ]:
r('''
    
    }
''') 